# Import

## Modules

In [20]:
%load_ext autoreload
%autoreload 2

from math import pi
import pandas as pd
import sys
from pathlib import Path
from dotenv import load_dotenv

import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Set path

In [2]:
PROJECT_ROOT = !git rev-parse --show-toplevel
PROJECT_ROOT = Path(PROJECT_ROOT[0])
PROJECT_ROOT

PosixPath('/workspaces/app.share_portfolio')

In [3]:
sys.path.append(str(PROJECT_ROOT))

## Modules

In [4]:
load_dotenv(str(PROJECT_ROOT / ".env"))
import src.data.make_dataset as md

## Data

In [5]:
portfolio = ["IVV", "CSL"]
data = md.get_portfolio(portfolio)


# Plot

In [6]:
data

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2021-02-01,375.17,378.7499,372.70,377.60,4393168.0,IVV
2021-01-29,377.06,378.0100,369.65,371.52,5001516.0,IVV
2021-01-28,377.76,383.3400,377.30,379.05,5677790.0,IVV
2021-01-27,381.63,381.7000,373.43,375.87,5592647.0,IVV
2021-01-26,386.81,387.2000,385.00,385.27,2219242.0,IVV
...,...,...,...,...,...,...
1999-11-05,32.06,32.7500,32.00,32.13,105200.0,CSL
1999-11-04,32.13,32.4400,31.75,31.94,191800.0,CSL
1999-11-03,33.00,33.1300,31.94,32.00,180300.0,CSL


In [49]:
# data.reset_index(inplace=True)
from bokeh.models import HoverTool
output_notebook()

mask = (data["Symbol"] == "IVV") & (data["Date"] > "2021-01-01")
subset = data[mask].dropna(subset=["Close"])
subset["Mean"] = (subset["Open"] + subset["Close"]) / 2.0

inc = subset["Close"] > subset["Open"]
dec = subset["Open"] > subset["Close"]
w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

# Candle Sticks
cs = figure(x_axis_type="datetime", tools=TOOLS, plot_width=700, plot_height=300, title = "MSFT Candlestick with Volume")
cs.xaxis.visible = False
cs.xaxis.major_label_orientation = pi/4
cs.grid.grid_line_alpha=0.3

cs.segment(subset["Date"], data["High"], subset["Date"], subset["Low"], color="black")
cs.vbar(subset.loc[inc, "Date"], w, subset.loc[inc, "Open"], subset.loc[inc, "Close"], fill_color="green", line_color="black")
cs.vbar(subset.loc[inc, "Date"], w, subset.loc[inc, "Open"], subset.loc[inc, "Close"], fill_color="green", line_color="black")
cs.vbar(subset.loc[dec, "Date"], w, subset.loc[dec, "Open"], subset.loc[dec, "Close"], fill_color="red", line_color="black")

# Format the tooltip
tooltips = [
            ('High', '@High'),
            ('Open','@Open'),
            ('Mean','@Mean'),
            ('Close', '@Close'),
            ('Low', '@Low'),
           ]

# Configure a renderer to be used upon hover
hover_glyph = cs.circle(x='Date', y='Mean', source=subset,
                         size=15, alpha=0,
                         hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
cs.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

# Volume
vol = figure(x_axis_type="datetime", tools="", toolbar_location=None, plot_width=700, plot_height=200, x_range=cs.x_range)
vol.xaxis.major_label_orientation = pi/4
vol.grid.grid_line_alpha=0.3
vol.vbar(subset["Date"], w, subset["Volume"], [0]*subset.shape[0])

show(gridplot([[cs],[vol]]))

Loading BokehJS ...

In [57]:
# data.reset_index(inplace=True)
from bokeh.models import HoverTool
output_notebook()

mask = (data["Symbol"] == "IVV") & (data["Date"] > "2021-01-01")
subset = data[mask].dropna(subset=["Close"])
subset["Mean"] = (subset["Open"] + subset["Close"]) / 2.0

inc = subset["Close"] > subset["Open"]
dec = subset["Open"] > subset["Close"]
w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

# Candle Sticks
cs = figure(x_axis_type="datetime", tools=TOOLS, plot_width=700, plot_height=300, title = "MSFT Candlestick with Volume")
cs.xaxis.visible = False
cs.xaxis.major_label_orientation = pi/4
cs.grid.grid_line_alpha=0.3

cs.segment(subset["Date"], data["High"], subset["Date"], subset["Low"], color="black")
cs.vbar(subset.loc[inc, "Date"], w, subset.loc[inc, "Open"], subset.loc[inc, "Close"], fill_color="green", line_color="black")
cs.vbar(subset.loc[inc, "Date"], w, subset.loc[inc, "Open"], subset.loc[inc, "Close"], fill_color="green", line_color="black")
cs.vbar(subset.loc[dec, "Date"], w, subset.loc[dec, "Open"], subset.loc[dec, "Close"], fill_color="red", line_color="black")

# Format the tooltip
tooltips = [
            ('High', '@High'),
            ('Open','@Open'),
            ('Mean','@Mean'),
            ('Close', '@Close'),
            ('Low', '@Low'),
           ]

# Configure a renderer to be used upon hover
hover_glyph = cs.circle(x='Date', y='Mean', source=subset,
                         size=15, alpha=0,
                         hover_fill_color='black', hover_alpha=0.5)

# Add the HoverTool to the figure
cs.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))

# Volume
vol = figure(x_axis_type="datetime", tools="", toolbar_location=None, plot_width=700, plot_height=200, x_range=cs.x_range)
vol.xaxis.major_label_orientation = pi/4
vol.grid.grid_line_alpha=0.3
vol.vbar(subset["Date"], w, subset["Volume"], [0]*subset.shape[0])

ds = cs.data_source

# create a callback that will reset the datasource
def callback(self):

    selected = [checkbox_group.labels[i] for i in checkbox_group.active]
    filtered =  subset[subset["Symbol"].isin(selected)]
    new_data = dict()
    new_x = filtered["Date"].unique()
    new_y = filtered.groupby("Date")["Value"].sum().values
    new_data["x"] = new_x
    new_data["y"] = new_y

    ds.data = new_data

# add checkboxes and the callback
labels = list(subset["Symbol"].unique())
checkbox_group = CheckboxGroup(labels=labels, active=[1])
checkbox_group.on_click(callback)

# put the checkboxes and plot in a layout and add to the document
curdoc().add_root(row(checkbox_group, p))

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 10-11: invalid continuation byte

In [30]:
data

,Date,Open,High,Low,Close,Volume,Symbol
0,2021-02-01,375.17,378.7499,372.70,377.60,4393168.0,IVV
1,2021-01-29,377.06,378.0100,369.65,371.52,5001516.0,IVV
2,2021-01-28,377.76,383.3400,377.30,379.05,5677790.0,IVV
3,2021-01-27,381.63,381.7000,373.43,375.87,5592647.0,IVV
4,2021-01-26,386.81,387.2000,385.00,385.27,2219242.0,IVV
...,...,...,...,...,...,...,...
10550,1999-11-05,32.06,32.7500,32.00,32.13,105200.0,CSL
10551,1999-11-04,32.13,32.4400,31.75,31.94,191800.0,CSL
10552,1999-11-03,33.00,33.1300,31.94,32.00,180300.0,CSL
10553,1999-11-02,32.75,33.0000,32.69,32.81,101300.0,CSL


# Example

src: https://stackoverflow.com/questions/64876637/plotting-candlestick-and-volume-candels-in-bokeh

In [ ]:
import bokeh
bokeh.sampledata.download()

from math import pi

import pandas as pd

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.sampledata.stocks import MSFT


In [3]:
output_notebook()

df = pd.DataFrame(MSFT)[:50]
df["date"] = pd.to_datetime(df["date"])

inc = df.close > df.open
dec = df.open > df.close
w = 12*60*60*1000 # half day in ms

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p1 = figure(x_axis_type="datetime", tools=TOOLS, plot_width=700, plot_height=300, title = "MSFT Candlestick with Volume")
p1.xaxis.visible = False
p1.xaxis.major_label_orientation = pi/4
p1.grid.grid_line_alpha=0.3

p1.segment(df.date, df.high, df.date, df.low, color="black")
p1.vbar(df.date[inc], w, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
p1.vbar(df.date[dec], w, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

p2 = figure(x_axis_type="datetime", tools="", toolbar_location=None, plot_width=700, plot_height=200, x_range=p1.x_range)
p2.xaxis.major_label_orientation = pi/4
p2.grid.grid_line_alpha=0.3
p2.vbar(df.date, w, df.volume, [0]*df.shape[0])

show(gridplot([[p1],[p2]]))

Loading BokehJS ...